In [2]:
import os, sys
from os.path import join

import nibabel as nib
import numpy as np
import pandas as pd

from nilearn.image import index_img
from nilearn.image import load_img
from nilearn.image import mean_img
from nilearn.image import new_img_like
from nilearn.masking import apply_mask

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Multivoxel approaches
This notebook explores whether we can build a multivoxel model for predicting the trial state (REST, MOTOR, IMAGERY) based on the first run of data, and then apply that model to subsequent runs. 


# Prep Inputs

In [5]:
# load timing files
trialOnsets = pd.read_csv('../data/subject001/timingFiles/trialOnsets.tsv', sep='\t')
trialOnsets

,onsets_TR1,onsets_TR2,trialTypes,weights,duration
0,4,2,rest,1.0,20
1,24,22,motor,1.0,10
2,34,32,rest,1.0,20
3,54,52,imagery,1.0,10
4,64,62,rest,1.0,20
5,84,82,imagery,1.0,10
6,94,92,rest,1.0,20
7,114,112,motor,1.0,10
8,124,122,rest,1.0,20
9,144,142,imagery,1.0,10


In [22]:
# mask that we will use to constrain the number of features
mask = 'motorSphere_5mm_FUNC_mask.nii.gz'
mask_img = load_img(join('../data/subject001/pyneal_002/mask_transforms/FUNC_masks', mask))

print('{} voxels'.format(int(sum(mask_img.get_data()))))

63 voxels


In [25]:
from nilearn.input_data import NiftiMasker

In [27]:
# set up a masker based on the mask
masker = NiftiMasker(mask_img=join('../data/subject001/pyneal_002/mask_transforms/FUNC_masks', mask), 
                     standardize=True)

fmri = masker.fit_transform('../data/subject001/pyneal_002/receivedFunc.nii.gz')
fmri.shape

(186, 63)